In [ ]:
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import shutil
import numpy as np
import tqdm

In [ ]:
from pycocotools.coco import COCO

In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5



Cloning into 'yolov5'...
remote: Enumerating objects: 3390, done.
remote: Total 3390 (delta 0), reused 0 (delta 0), pack-reused 3390
Receiving objects: 100% (3390/3390), 7.02 MiB | 32.11 MiB/s, done.
Resolving deltas: 100% (2276/2276), done.
/content/yolov5/yolov5


In [ ]:
!git clone 'https://github.com/mhrah7495/TACO'

Cloning into 'TACO'...
remote: Enumerating objects: 309, done.
remote: Total 309 (delta 0), reused 0 (delta 0), pack-reused 309
Receiving objects: 100% (309/309), 59.13 MiB | 34.84 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [ ]:
!pip3 install -r TACO/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import zipfile
from google.colab import drive

!pip install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#extract dataset

!unzip '/content/drive/MyDrive/taco.zip' -d 'tacotrashdataset'

Archive:  /content/drive/MyDrive/taco.zip
   creating: tacotrashdataset/batch_1/
   creating: tacotrashdataset/batch_10/
   creating: tacotrashdataset/batch_11/
   creating: tacotrashdataset/batch_12/
   creating: tacotrashdataset/batch_13/
   creating: tacotrashdataset/batch_14/
   creating: tacotrashdataset/batch_15/
   creating: tacotrashdataset/batch_2/
   creating: tacotrashdataset/batch_3/
   creating: tacotrashdataset/batch_4/
   creating: tacotrashdataset/batch_5/
   creating: tacotrashdataset/batch_6/
   creating: tacotrashdataset/batch_7/
   creating: tacotrashdataset/batch_8/
   creating: tacotrashdataset/batch_9/
  inflating: tacotrashdataset/all_image_urls.csv  
  inflating: tacotrashdataset/annotations_unofficial.json  
  inflating: tacotrashdataset/annotations.json  
  inflating: tacotrashdataset/batch_5/000093.JPG  
  inflating: tacotrashdataset/batch_5/000098.JPG  
  inflating: tacotrashdataset/batch_5/000031.JPG  
  inflating: tacotrashdataset/batch_5/000108.JPG  
  i

In [ ]:
data_source = COCO(annotation_file='/content/yolov5/tacotrashdataset/annotations.json')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [ ]:
# remapping label id to 0~1
label_transfer = {5: 0, 12: 1}


In [ ]:
img_ids = data_source.getImgIds()

catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])
classes = {}
coco_labels = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

class_num = {}

In [ ]:
!mkdir -p tmp/labels tmp/images
save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

In [ ]:
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    img_info = data_source.loadImgs(img_id)[0]
    
    save_name = img_info['file_name'].replace('/', '_')
    
    file_name = save_name.split('.')[0]
    
    height = img_info['height']
    width = img_info['width']
    
    save_path = save_base_path + file_name + '.txt'
    is_exist = False  
    with open(save_path, mode='w') as fp:
        
        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0:  
            fp.write('')
            continue
        
        annotations = data_source.loadAnns(annotation_id)
        lines = ''  
        
        for annotation in annotations:
            
            label = coco_labels_inverse[annotation['category_id']]
            if label in label_transfer.keys():
                
                is_exist = True
                box = annotation['bbox']
                if box[2] < 1 or box[3] < 1:
                    
                    continue
                # top_x,top_y,width,height==>cen_x,cen_y,width,height
                box[0] = round((box[0] + box[2] / 2) / width, 6)
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / height, 6)
                label = label_transfer[label]  
                if label not in class_num.keys():
                    class_num[label] = 0
                class_num[label] += 1
                lines = lines + str(label)  
                for i in box:  
                    lines += ' ' + str(i)
                lines += '\n'  
        fp.writelines(lines)
    if is_exist:
        
        shutil.copy('/content/yolov5/tacotrashdataset/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
        
        os.remove(save_path)

change .json file to .txt file: 101it [00:00, 347.76it/s]


FileNotFoundError: ignored

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('tmp', output="taco", seed=1337, ratio=(.8, 0.1,0.1)) 

In [ ]:
print(sorted(os.listdir('taco/train/images'))[:5])

In [ ]:
from google.colab import files

In [ ]:
!pip install -U PyYAML

In [ ]:
!pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!python train.py --img 320 --batch 1 --epochs 3 --data /content/yolov5/taco2.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights /content/yolov5/taco2_epoch300_best.pt

In [ ]:
!ls taco/test/images

In [ ]:
!cp /content/yolov5/runs/train/exp/weights/best.pt weights

In [ ]:
from IPython.display import Image

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/taco/test/images/batch_1_000000.jpg

In [ ]:
Image('/content/yolov5/runs/detect/exp/000008.jpg')

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/taco/test/images/batch_5_000017.JPG 

In [ ]:
Image('/content/yolov5/tacotrashdataset/batch_5/000004.JPG')

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt  --img 320 --conf 0.4 --source /content/yolov5/taco/test/images

In [ ]:
Image('/content/yolov5/tacotrashdataset/batch_4/000008.JPG')

In [ ]:
Image('/content/yolov5/tacotrashdataset/batch_14/000006.jpg')

In [ ]:
Image('runs/detect/exp3/batch_8_000027.jpg')